In [24]:
import random
import math
import matplotlib.pyplot as plt
import time
import numpy as np
from point import Point
import json

# Load the Layout Pool from Static Generation

In [31]:
with open("data/static_layout_pool.json", 'r') as openfile:
    json_object = json.load(openfile)

WIDTH = json_object["width"]
LENGTH = json_object["length"]
layouts = [[Point(p[0], p[1]) for p in lay] for lay in json_object["layouts"]]

# Dynamic Trial Shuffling
- We shuffle the N trials such that the boundary visibility alternates each time.
- We read N/2 trials from static generation.
- Boundary visibility is alternating, so we simply assume border_visibility = i % 2

## Generation

- At least K = 5 trials between trials of the same pairing.
- We make sure that between adjacent pairs, the ending location and starting location is always opposite.
    - Also, make sure these distances obey MIN_DIST_PATH

In [60]:
NUM_SHUFFLES = 50
DIST_BETWEEN_COPY = 5 # K
MIN_DIST_PATH = 1.5
LAYOUT_COUNT = len(layouts)

In [64]:
def generate_random_shuffle():
    # we interleave to random lists of 1...n/2, therefore it is guaranteed that the random shuffling
    #is valid for the constraint: boundary visibility alternates each time. each layout gets exactly 
    #one case with border, exactly one without border
    def random_half_shuffle():
        l = list(range(len(layouts)))
        random.shuffle(l)
        return l
    
    l = [x for t in zip(random_half_shuffle(), random_half_shuffle()) for x in t]
    return l

def is_valid_shuffle(order):
    # At least K = 5 trials between trials of the same pairing.
    # since we generate this as half and half, we only need to check the first and last K elements
    K = DIST_BETWEEN_COPY
    for i in range(K, len(order)):
        if order[i] in order[i-K:i]:
            #print("----", i, K, order[i], order[i-K:i])
            return False
    
    # each pair gets one occurence with border, one occurence without border
    # as a property of generate_random_shuffle, therefore we do not test for it.
    
    #TODO: We make sure that between adjacent pairs, the ending location and starting location is always opposite.
    #TODO: Also, make sure these distances obey MIN_DIST_PATH
    
    rule_adjacentPairNotSameSide_violations = 0
    rule_minDistPath_violations = 0
    for i in range(len(order) - 1):
        pre_point = layouts[order[i]][-1]
        post_point = layouts[order[i+1]][0]
        
        pre_side = pre_point.y < LENGTH/2
        post_side = post_point.y < LENGTH/2
        
        if pre_side == post_side:
            rule_adjacentPairNotSameSide_violations += 1
            
        if pre_point.dist(post_point) < MIN_DIST_PATH:
            rule_minDistPath_violations += 1
        
    print("*" * 15,rule_adjacentPairNotSameSide_violations, "-" * 5 , rule_minDistPath_violations, "*"*15)
    # rule_adjacentPairNotSameSide_violations -> could we mirror the layouts to satisfy this constraint?
    # rule_minDistPath_violations -> could we mirror on x-axis or maybe a acceptable lower bound?
    return True
        
def generate_valid_shuffle():
    shuffle = generate_random_shuffle()
    print(shuffle)
    while not is_valid_shuffle(shuffle):
        shuffle = generate_random_shuffle()
    return shuffle

In [65]:
shuffles = [generate_valid_shuffle() for i in range(1)]
shuffles

[17, 47, 43, 45, 39, 48, 4, 30, 10, 10, 35, 25, 32, 23, 2, 28, 45, 46, 48, 36, 22, 41, 46, 9, 1, 13, 36, 49, 33, 34, 26, 2, 12, 35, 14, 32, 40, 12, 16, 33, 3, 17, 27, 0, 29, 7, 19, 38, 41, 16, 6, 3, 21, 40, 7, 11, 38, 18, 9, 4, 15, 14, 49, 31, 28, 39, 44, 26, 23, 8, 8, 1, 25, 27, 24, 42, 42, 24, 37, 43, 30, 19, 13, 6, 47, 15, 20, 22, 31, 37, 0, 20, 5, 5, 34, 21, 18, 44, 11, 29]
*************** 43 ----- 8 ***************


[[12,
  6,
  46,
  38,
  41,
  24,
  11,
  4,
  40,
  8,
  35,
  21,
  15,
  10,
  32,
  45,
  23,
  28,
  26,
  46,
  18,
  34,
  1,
  20,
  47,
  35,
  24,
  32,
  14,
  42,
  9,
  43,
  39,
  25,
  13,
  30,
  3,
  0,
  33,
  36,
  22,
  14,
  37,
  23,
  21,
  3,
  6,
  1,
  43,
  17,
  36,
  47,
  4,
  13,
  19,
  37,
  2,
  41,
  31,
  18,
  38,
  29,
  28,
  15,
  27,
  5,
  49,
  48,
  30,
  16,
  44,
  27,
  42,
  7,
  45,
  22,
  8,
  11,
  48,
  33,
  10,
  19,
  20,
  2,
  17,
  49,
  16,
  9,
  7,
  44,
  5,
  40,
  29,
  31,
  34,
  26,
  0,
  39,
  25,
  12]]

## Checks

- We make sure that apmprox. half the pairs happen in A->B, and the other half happens in B->A
- We want to check the number of objects within the same side across a sliding window of 5 trials.

# Objects
- 5 different shapes (circle, cube, cylinder, pyramid, star)
- 5 different colors from a color-blind-friendly palette.
-> 25 different combinations.

Any object generated must not have been in the last 10 shown object.